## Libs

In [ ]:
! pip install sentence_transformers

In [ ]:
!pip install umap-learn

!pip install hdbscan

     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=4ddc1d2d228d98c9d271e9603966105b3858cdc6fcf0735062ebcf9ba56cb36e
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=333c3ceef2f6a1d4ea5ea008042fc71db4517daceca7768054e77ede7ee2ba4f
  Stored in directory: /root/.cache/pip/wheels/af/e9/33/04db1436df0757c42fda8ea6796d7a8586e23c85fac355f476
Successfully built umap-learn pynndescent
     |████████████████████████████████| 6.4 MB 13.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311933 sha256=1699e33f6477183f5c707fbe91d

In [ ]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import umap.umap_ as umap
import plotly.express as px
import pandas as pd

In [ ]:
import json
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import re
import torch
from tqdm import tqdm


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data reading

In [ ]:
ls "drive/MyDrive/Colab Notebooks"

 Alla_Chepurova_PMLDL_Midterm.ipynb   ICV/                    PMLDL/
'Control Theory Homeworks'/          'IML HW'/               'PMLDL Project'/
'DLS F20'/                           'IR HW'/                 SIML/
'DSP HW'/                            'Mckinsey Hack S2021'/


In [ ]:
! ls "drive/MyDrive/Colab Notebooks/PMLDL Project"

 books.zip			     embed_books_with_word2vec.ipynb.ipynb
 data.json			     sbert_embedded_data.json
'Embed_books_with_sbert (1).ipynb'   txt_to_json.ipynb
 Embed_books_with_sbert.ipynb


In [ ]:
data = {}
with open('drive/MyDrive/Colab Notebooks/PMLDL Project/data.json', 'r') as f:
    data = json.load(f)
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
data['books'][0]

{'author': 'George Orwell',
 'description': ' Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, \'Beasts of England\'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase 

In [ ]:
genres = []
for b in data['books']:
  for g in b['genres']:
    genres.append(g)

list(set(genres))

['Zombie',
 'Colonial United States romance',
 'Humour',
 'Apocalyptic and post-apocalyptic fiction',
 'Comic science fiction',
 'Comedy of manners',
 'Locked room mystery',
 'Psychology',
 'Neuroscience',
 'Autobiographical novel',
 'New York Times Best Seller list',
 'Sword and planet',
 'Invasion literature',
 'Albino bias',
 'Comic book',
 'Techno-thriller',
 'Roman à clef',
 'Literary fiction',
 'Wuxia',
 'Alien invasion',
 'Georgian romance',
 'Zombies in popular culture',
 'New Weird',
 'Subterranean fiction',
 'Vampire fiction',
 'Social novel',
 'Treatise',
 'Religion',
 'Memoir',
 "Boys' school stories",
 'Sports',
 'Biographical novel',
 'Science fantasy',
 'Edisonade',
 'Photography',
 'Historical romance',
 'Non-fiction',
 'Play',
 'Detective fiction',
 'Prose poetry',
 'Travel',
 'Western fiction',
 'Cabal',
 'Education',
 'Epistolary novel',
 'Historical fiction',
 'Social science fiction',
 'Anti-nuclear',
 'Political philosophy',
 'Future history',
 'Nature',
 'Satire'

## Model testing

In [ ]:
sbert_model = SentenceTransformer('all-mpnet-base-v2',  device='cuda')
sentences = ["I ate dinner.", 
       "We had a three-course meal.", 
       "Brad came to dinner with us.",
       "He loves fish tacos.",
       "In the end, we all felt like we ate too much.",
       "We all agreed; it was a magnificent evening."]
sentence_embeddings = sbert_model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])
sentence_embeddings.shape

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
query = "I had pizza and pasta"
query_vec = sbert_model.encode([query])[0]
query_vec.shape

(768,)

In [ ]:
for sent in sentences:
  sim = cosine(query_vec, sbert_model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  I ate dinner. ; similarity =  0.6722933
Sentence =  We had a three-course meal. ; similarity =  0.5845446
Sentence =  Brad came to dinner with us. ; similarity =  0.426609
Sentence =  He loves fish tacos. ; similarity =  0.2538456
Sentence =  In the end, we all felt like we ate too much. ; similarity =  0.4099004
Sentence =  We all agreed; it was a magnificent evening. ; similarity =  0.29891813


## Functions for book preprocessing

In [ ]:
def tokenize_book(book_description, tokenizer):
  sentences =  tokenizer(book_description)
  sentences = [ re.sub(' +', ' ', x) for x in sentences]
  return sentences

In [ ]:
book_sents = tokenize_book(data["books"][0]["description"], sent_tokenize)
len(book_sents)

52

In [ ]:
def embed_book(book_sentences, model):
  embs = torch.tensor(model.encode(book_sentences))
  emb = torch.mean(embs, axis=0)
  return emb

In [ ]:
emb = embed_book(book_sents, sbert_model)
emb.shape

torch.Size([768])

## Adding ids to json to put in bd afterward

In [ ]:
def enumerate_books(data):
  new_books = []
  for i, book in enumerate(data["books"]):
    new_book = {}
    new_book["id"] = i
    for key in book:
      new_book[key] = book[key]    
    new_books.append(new_book)
  return new_books

In [ ]:
books = enumerate_books(data)
books

In [ ]:
data["books"] = books

## Embedding all of the book in the dataset, saving it to dictionary and same json to put it into bd afterward

In [ ]:
def embed_book_dataset(books, tokenizer, model):
  embedded_books = []

  for book in tqdm(books):
    embedded_book = {}
    embedded_book["id"] = book["id"]

    # tokenize description
    book_sents = tokenize_book(book["description"], tokenizer)
    # embed description
    emb = np.array(embed_book(book_sents, model))

    embedded_book["book_vector"] = emb
    embedded_books.append(embedded_book)
  return embedded_books


In [ ]:
embedded_books = embed_book_dataset(data["books"], sent_tokenize, sbert_model)

100%|██████████| 16559/16559 [42:52<00:00,  6.44it/s]


In [ ]:
for i, book in enumerate(embedded_books):
  embedded_books[i]["book_vector"] =  book["book_vector"].astype(object).tolist()

In [ ]:
embedded_books[0]

In [ ]:
data["embedded_books"] = embedded_books

In [ ]:
! touch 'drive/MyDrive/Colab Notebooks/PMLDL Project/sber_embedded_data.json'

In [ ]:
with open('drive/MyDrive/Colab Notebooks/PMLDL Project/sber_embedded_data.json', 'w') as f:
    json.dump(data, f)

In [ ]:
new_data = {}
with open('drive/MyDrive/Colab Notebooks/PMLDL Project/sber_embedded_data.json', 'r') as f:
    new_data = json.load(f)


In [ ]:
embeddings = [x["book_vector"] for x in embedded_books]

In [ ]:
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning:

The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.



In [ ]:
result = pd.DataFrame(umap_data, columns=['x', 'y'])

In [ ]:
result["titles"] = [x["title"] for x in data["books"]]

In [ ]:
fig = px.scatter(result.iloc[:100, :], x='x', y='y', text='titles')
fig.show()

In [ ]:
fig = px.scatter(result, x='x', y='y', text='titles')
fig.show()